In [ ]:
%%capture

# 安装 unsloth 包。unsloth 是一个用于微调大型语言模型（LLM）的工具，可以让模型运行更快、占用更少内存。
!pip install unsloth

# 卸载当前已安装的 unsloth 包（如果已安装），然后从 GitHub 的源代码安装最新版本。
# 这样可以确保我们使用的是最新功能和修复。
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

# 安装 bitsandbytes 和 unsloth_zoo 包。
# bitsandbytes 是一个用于量化和优化模型的库，可以帮助减少模型占用的内存。
# unsloth_zoo 可能包含了一些预训练模型或其他工具，方便我们使用。
!pip install bitsandbytes unsloth_zoo

In [ ]:
from unsloth import FastLanguageModel  # 导入FastLanguageModel类，用来加载和使用模型
import torch  # 导入torch工具，用于处理模型的数学运算

max_seq_length = 1024  # 设置模型处理文本的最大长度，相当于给模型设置一个“最大容量”
dtype = None  # 设置数据类型，让模型自动选择最适合的精度
load_in_4bit = True  # 使用4位量化来节省内存，就像把大箱子压缩成小箱子

# 加载预训练模型，并获取tokenizer工具
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B",  # 指定要加载的模型名称
    max_seq_length=max_seq_length,  # 使用前面设置的最大长度
    dtype=dtype,  # 使用前面设置的数据类型
    load_in_4bit=load_in_4bit,  # 使用4位量化
    # token="hf_...",  # 如果需要访问授权模型，可以在这里填入密钥
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.6: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
prompt_style = """以下是描述任务的指令，以及提供进一步上下文的输入。
请写出一个适当完成请求的回答。
在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。

### 指令：
你是一位精通卜卦、星象和运势预测的算命大师。
请回答以下算命问题。

### 问题：
{}

### 回答：
<think>{}"""
# 定义提示风格的字符串模板，用于格式化问题

question = "1992年闰四月初九巳时生人，女，想了解健康运势"
# 定义具体的算命问题

In [ ]:
FastLanguageModel.for_inference(model)
# 准备模型以进行推理

inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")
# 使用 tokenizer 对格式化后的问题进行编码，并移动到 GPU

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
# 使用模型生成回答

response = tokenizer.batch_decode(outputs)
# 解码模型生成的输出为可读文本

print(response[0])
# 打印生成的回答部分

<｜begin▁of▁sentence｜>以下是描述任务的指令，以及提供进一步上下文的输入。
请写出一个适当完成请求的回答。
在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。

### 指令：
你是一位精通卜卦、星象和运势预测的算命大师。
请回答以下算命问题。

### 问题：
1992年闰四月初九巳时生人，女，想了解健康运势

### 回答：
<think>
好的，我现在需要帮一位1992年闰四月初九巳时出生的女性分析她的健康运势。首先，我要回顾她的生日、时间和性别，这些都是算命的基础信息。闰四月初九巳时，属于春季，生肖是牛，可能带来一些特定的运势。

接下来，我会查看她的生日对应的五行元素。1992年是金属年，属金，而她的名字“小芳”发音可能带有金属的意味，但可能需要进一步确认。她的名字中的“芳”发音可能与木有关，因为“芳”在古代可能与木有关，木为春季，代表生机和希望。

然后，我会考虑她的健康方面。由于她出生在闰四月初九巳时，这个时间段可能带来一些身体上的特质。巳时在中国传统时钟中是午时，代表阳光和活力，所以她可能身体健康，充满活力，但需要注意过度劳累和阳光过强的问题。

她的健康运势方面，可能会有一定的挑战，比如容易受感冒或风湿病影响，或者在身体方面有轻微的不适。为了缓解这些问题，她可以通过调整作息，保持规律的作息时间，避免熬夜，多休息，多喝温水，保持身体的水分和能量。

此外，她可能需要关注自己的情绪管理，因为情绪波动可能影响身体健康。保持积极的心态，学会放松和放松，进行适当的运动，如散步、瑜伽或轻度运动，可以帮助维持健康。

她的五行平衡方面，金和木可能稍微失衡，建议她适当进行绿色植物的养护，如养一盆绿色植物，或者在家中摆放绿色植物，以平衡木火的能量，促进家庭的和谐与健康。

总结一下，她的健康运势总体上是平稳的，但需要注意一些常见的身体问题，通过合理的生活方式和健康习惯来维护身体健康，保持积极的心态，是关键。
</think>

您好，根据您1992年闰四月初九巳时的出生信息以及您是女性的身份，我可以为您进行健康运势的分析。

首先，了解您的五行元素。1992年是金年，而您的名字“小芳”发音可能带有金属的意味，可能与金有关。根据中国传统的五行理论，金代表火，火代表活力和能量。因此，可能在身体上表现出较强的活力和能量，但需要

In [ ]:
# 定义一个用于格式化提示的多行字符串模板
train_prompt_style = """以下是描述任务的指令，以及提供进一步上下文的输入。
请写出一个适当完成请求的回答。
在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。

### 指令：
你是一位精通八字算命、 紫微斗数、 风水、易经卦象、塔罗牌占卜、星象、面相手相和运势预测等方面的算命大师。
请回答以下算命问题。

### 问题：
{}

### 回答：
<思考>
{}
</思考>
{}"""

In [ ]:
# 定义结束标记（EOS_TOKEN），用于指示文本的结束
EOS_TOKEN = tokenizer.eos_token  # 必须添加结束标记

# 导入数据集加载函数
from datasets import load_dataset
# 加载指定的数据集，选择中文语言和训练集的前500条记录
dataset = load_dataset("Conard/fortune-telling", 'default', split = "train[0:200]", trust_remote_code=True)
# 打印数据集的列名，查看数据集中有哪些字段
print(dataset.column_names)

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

all_details.json:   0%|          | 0.00/693k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/207 [00:00<?, ? examples/s]

['Question', 'Response', 'Complex_CoT']


In [ ]:
# 定义一个函数，用于格式化数据集中的每条记录
def formatting_prompts_func(examples):
    # 从数据集中提取问题、复杂思考过程和回答
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []  # 用于存储格式化后的文本
    # 遍历每个问题、思考过程和回答，进行格式化
    for input, cot, output in zip(inputs, cots, outputs):
        # 使用字符串模板插入数据，并加上结束标记
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)  # 将格式化后的文本添加到列表中
    return {
        "text": texts,  # 返回包含所有格式化文本的字典
    }

dataset = dataset.map(formatting_prompts_func, batched = True)
dataset["text"][0]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

'以下是描述任务的指令，以及提供进一步上下文的输入。\n请写出一个适当完成请求的回答。\n在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。\n\n### 指令：\n你是一位精通八字算命、 紫微斗数、 风水、易经卦象、塔罗牌占卜、星象、面相手相和运势预测等方面的算命大师。\n请回答以下算命问题。\n\n### 问题：\n新房装修,大门对着电梯好不好?要如何化解?\n\n### 回答：\n<思考>\n好的，用户问的是新房装修时大门对着电梯好不好，以及如何化解。首先，我需要回忆一下风水学中关于大门和电梯的相关知识。电梯在风水中属于动气比较强的地方，因为电梯频繁开合，会带来不稳定的气流，也就是所谓的“煞气”。大门是住宅的纳气口，如果正对电梯，可能会让这些不稳定的气流直接冲进家里，影响居住者的健康和财运。\n\n接下来，我需要确认用户的具体情况。比如，大门和电梯的距离有多远？是否正对还是稍微偏一点？不过用户没有提供这些细节，所以只能给出一般性的建议。化解的方法通常有几种：屏风或玄关、门帘、五帝钱、植物、八卦镜等。需要逐一解释这些方法的原理和使用方式，同时提醒用户要根据实际情况选择，必要时咨询专业风水师。\n\n另外，还要注意语气要亲切，避免使用过于专业的术语，让用户容易理解。同时，要强调这些是传统方法，效果因人而异，保持客观中立。最后，可以建议用户如果情况复杂，最好请专业人士实地查看，这样更稳妥。\n\n</思考>\n根据传统风水学的观点，大门正对电梯易形成"开口煞"，电梯频繁升降会扰乱家宅气场。建议化解方案：\n\n1. 玄关阻隔法\n在入门处设置L型屏风或文化砖玄关墙，高度以1.8米为宜，既保持采光又形成缓冲带\n\n2. 五行通关法\n门槛石下埋设五帝钱+白玉葫芦，建议选丙申年铸造的真品古币，配合门楣悬挂九宫八卦镜\n\n3. 光影化解术\n安装磨砂玻璃内推门，门框镶嵌黄铜门槛，每日辰时用海盐净化门廊区域\n\n4. 现代科技方案\n入户区安装智能感应灯带，设置循环播放的流水声效，运用声光电技术平衡磁场\n\n需注意电梯井方位与家主命卦的关系，建议提供具体户型平面图进行吉凶方位测算。当代建筑中可采用半透明艺术隔断结合空气净化系统，既符合科学原理又兼顾传统智慧。<｜end▁of▁sentence｜>'

第五步 执行微调

In [ ]:
FastLanguageModel.for_training(model)

model = FastLanguageModel.get_peft_model(
    model,  # 传入已经加载好的预训练模型
    r = 8,  # 设置 LoRA 的秩，决定添加的可训练参数数量
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",  # 指定模型中需要微调的关键模块
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 8,  # 设置 LoRA 的超参数，影响可训练参数的训练方式
    lora_dropout = 0,  # 设置防止过拟合的参数，这里设置为 0 表示不丢弃任何参数
    bias = "none",    # 设置是否添加偏置项，这里设置为 "none" 表示不添加
    use_gradient_checkpointing = "unsloth",  # 使用优化技术节省显存并支持更大的批量大小
    random_state = 3407,  # 设置随机种子，确保每次运行代码时模型的初始化方式相同
    use_rslora = False,  # 设置是否使用 Rank Stabilized LoRA 技术，这里设置为 False 表示不使用
    loftq_config = None,  # 设置是否使用 LoftQ 技术，这里设置为 None 表示不使用
)

Unsloth 2025.5.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
from trl import SFTTrainer  # 导入 SFTTrainer，用于监督式微调
from transformers import TrainingArguments  # 导入 TrainingArguments，用于设置训练参数
from unsloth import is_bfloat16_supported  # 导入函数，检查是否支持 bfloat16 数据格式

trainer = SFTTrainer(  # 创建一个 SFTTrainer 实例
    model=model,  # 传入要微调的模型
    tokenizer=tokenizer,  # 传入 tokenizer，用于处理文本数据
    train_dataset=dataset,  # 传入训练数据集
    dataset_text_field="text",  # 指定数据集中文本字段的名称
    max_seq_length=max_seq_length,  # 设置最大序列长度
    dataset_num_proc=2,  # 设置数据处理的并行进程数
    packing=False,  # 是否启用打包功能（这里设置为 False，打包可以让训练更快，但可能影响效果）
    args=TrainingArguments(  # 定义训练参数
        per_device_train_batch_size=1,  # 每个设备（如 GPU）上的批量大小
        gradient_accumulation_steps=8,  # 梯度累积步数，用于模拟大批次训练
        warmup_steps=5,  # 预热步数，训练开始时学习率逐渐增加的步数
        max_steps=75,  # 最大训练步数
        learning_rate=2e-4,  # 学习率，模型学习新知识的速度
        fp16=not is_bfloat16_supported(),  # 是否使用 fp16 格式加速训练（如果环境不支持 bfloat16）
        bf16=is_bfloat16_supported(),  # 是否使用 bfloat16 格式加速训练（如果环境支持）
        logging_steps=1,  # 每隔多少步记录一次训练日志
        optim="adamw_8bit",  # 使用的优化器，用于调整模型参数
        weight_decay=0.01,  # 权重衰减，防止模型过拟合
        lr_scheduler_type="linear",  # 学习率调度器类型，控制学习率的变化方式
        seed=3407,  # 随机种子，确保训练结果可复现
        output_dir="outputs",  # 训练结果保存的目录
        report_to="none",  # 是否将训练结果报告到外部工具（如 WandB），这里设置为不报告
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 200 | Num Epochs = 3 | Total steps = 75
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 20,971,520/8,000,000,000 (0.26% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.340700
2,2.245600
3,2.469700
4,2.224500
5,2.088900
6,2.314000
7,2.074700
8,2.087100
9,1.936500
10,1.963100


In [ ]:
print(question) # 打印前面的问题

1992年闰四月初九巳时生人，女，想了解健康运势


In [ ]:
# 将模型切换到推理模式，准备回答问题
FastLanguageModel.for_inference(model)

# 将问题转换成模型能理解的格式，并发送到 GPU 上
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# 让模型根据问题生成回答，最多生成 4000 个新词
outputs = model.generate(
    input_ids=inputs.input_ids,  # 输入的数字序列
    attention_mask=inputs.attention_mask,  # 注意力遮罩，帮助模型理解哪些部分重要
    max_new_tokens=4000,  # 最多生成 4000 个新词
    use_cache=True,  # 使用缓存加速生成
)

# 将生成的回答从数字转换回文字
response = tokenizer.batch_decode(outputs)

# 打印回答
print(response[0])

<｜begin▁of▁sentence｜>以下是描述任务的指令，以及提供进一步上下文的输入。
请写出一个适当完成请求的回答。
在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。

### 指令：
你是一位精通卜卦、星象和运势预测的算命大师。
请回答以下算命问题。

### 问题：
1992年闰四月初九巳时生人，女，想了解健康运势

### 回答：
<think>

<思考>
好的，用户提供了一个具体的生辰信息，需要根据传统命理来分析健康运势。首先，需要确认用户提供的具体日期和时间是否正确，或者是否有需要补充的信息。例如，用户提到“闰四月初九巳时”，但需要确认具体的年份和闰年的情况。

接下来，需要将生辰转化为八字命盘，确定大运、流年和财帛等宫位的分布。然后，根据命盘中涉及健康的五行、五方、五神（如天、地、火、水、金、天、命、运、祿、祱）以及对应的健康宫位（比如伤官宫、左辅宫等），分析健康运势的主要组合和影响。

还需要考虑当前的大运流年，查看是否有健康相关的吉星、凶星或变动。例如，流年是否有“紫微斗数”、“天同”等吉星，或者有“陀罗”、“陀罗星”等凶星影响。如果有流年吉星，可能健康运势较好；若有凶星，可能需要注意健康问题。

另外，要参考命盘中的五行分布。健康运势受五行平衡影响，若命盘中五行过旺或过弱，可能需要通过调整饮食、习惯等方式来调和。例如，火过旺可能需要多补水，水过旺则需要多补木。

还需结合生辰中的流年、世运和人事运势，分析健康问题的出现时间和可能的解决方式。例如，如果当前大运有“伤官”吉星，可能需要注意心血管系统；若有“左辅”吉星，可能关注肠胃健康。

最后，结合传统命理中的“天时地理”理论，分析生辰中天时因素对健康的影响。例如，阴阳气血的强弱、五行的分布等。

需要注意的是，传统命理中健康运势只是参考，实际健康状况还需结合现代医学进行综合判断。用户应结合自身健康状况和医生建议，进行科学的健康管理。

</思考>
**1992年闰四月初九巳时（甲寅年，闰四月初九）生人女健康运势分析**

**基础信息验证**  
需确认具体年份与闰年匹配（1992年是甲寅年，闰四月需查实闰日），时间为巳时（9:00-11:00），若有其他时间需补充。建议补充生辰完整（出生日期、时间、性别）。

---

### **命盘基础分析**
**八字

In [ ]:
question = "2001年正月初十戌时生人，男，想了解情感运势"

In [ ]:
# 将模型切换到推理模式，准备回答问题
FastLanguageModel.for_inference(model)

# 将问题转换成模型能理解的格式，并发送到 GPU 上
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# 让模型根据问题生成回答，最多生成 4000 个新词
outputs = model.generate(
    input_ids=inputs.input_ids,  # 输入的数字序列
    attention_mask=inputs.attention_mask,  # 注意力遮罩，帮助模型理解哪些部分重要
    max_new_tokens=4000,  # 最多生成 4000 个新词
    use_cache=True,  # 使用缓存加速生成
)

# 将生成的回答从数字转换回文字
response = tokenizer.batch_decode(outputs)

# 打印回答
print(response[0])

<｜begin▁of▁sentence｜>以下是描述任务的指令，以及提供进一步上下文的输入。
请写出一个适当完成请求的回答。
在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。

### 指令：
你是一位精通卜卦、星象和运势预测的算命大师。
请回答以下算命问题。

### 问题：
2001年正月初十戌时生人，男，想了解情感运势

### 回答：
<think>
<思考>
好的，用户提供了一个生辰：2001年正月初十戌时，男性，想了解情感运势。首先，我需要确认这个生辰对应的八字。2001年是戊寅年，正月初十是己卯日，戌时是亥月。需要查出具体的八字排盘，包括年柱、月柱、日柱和时柱。

接下来，分析八字中的五行分布。戊寅年属木，己卯日属土，亥月属水，戌时属土。土旺，木生土，水生木，形成土木相生的关系。五行平衡上，土旺，木生，水生，火无，金无。需要看是否有吉凶神煞的影响，比如是否有火土相克，或者有吉星如天府、文昌等。

然后，考虑用户的性别是男性，情感运势通常与桃花、婚姻、人际关系相关。需要结合八字中的桃花星、婚姻星，以及流年和流月的影响。比如，2001年己卯年是桃花年，可能有机遇，但需要看八字中的桃花星是否旺盛。

另外，八字中的煞星位置也很重要。比如，如果有火星在戌时，可能会带来冲动或不稳定的情感。需要综合分析这些因素，给出建议，比如是否建议稳定人际关系，避免冲动行动，或者适当增强木火来平衡五行。

还要考虑用户的具体问题，情感运势可能包括感情发展、婚姻状况、人际关系等。需要结合八字中的流年和流月，比如2024年甲辰年是否有桃花，是否有贵人星出现，是否有财运带来的压力等。

最后，整理这些信息，给出一个连贯的分析，既有八字的基础，又结合流年和流月的影响，最后给出建议。注意用语要通俗易懂，避免过于专业的术语，让用户容易理解。

</思考>
**八字排盘与五行分析**  
2001年戊寅年（木寅）  
正月初十己卯日（土卯）  
亥月戌时（水戌）  

**五行分布**  
- 戊寅年：木旺  
- 己卯日：土旺  
- 亥月戌时：土旺水生  
- 木生土，水生木，土旺水生，形成土木相生关系  

**吉凶神煞**  
- 土旺木生，水生木，土旺水生，火无金无  
- 戌月戌时土旺，若有火星入，土火相克，需注意情感冲动  
- 己卯日天府星入，桃

In [ ]:
# 导入 Google Colab 的 userdata 模块，用于访问用户数据
from google.colab import userdata

# 从 Google Colab 用户数据中获取 Hugging Face 的 API 令牌
HUGGINGFACE_TOKEN = userdata.get('HUGGINGFACE_TOKEN')

# 将模型保存为 4 位量化格式（q4_k_m）
# 4 位量化文件最小，但精度可能稍低
if True: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.
Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 6.0G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 6.08 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 50%|█████     | 16/32 [00:02<00:01,  9.18it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [02:32<00:00,  4.78s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model/pytorch_model-00001-of-00004.bin...
Unsloth: Saving model/pytorch_model-00002-of-00004.bin...
Unsloth: Saving model/pytorch_model-00003-of-00004.bin...
Unsloth: Saving model/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at model into f16 GGUF format.
The output location will be /content/model/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf:

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### We removed it in GGUF's chat template for you.


Unsloth: Conversion completed! Output location: /content/model/unsloth.Q4_K_M.gguf


In [ ]:
# 导入 Hugging Face Hub 的 create_repo 函数，用于创建一个新的模型仓库
from huggingface_hub import create_repo

# 在 Hugging Face Hub 上创建一个新的模型仓库
create_repo("Hextalk4bit", token=HUGGINGFACE_TOKEN, exist_ok=True)

# 将模型和分词器上传到 Hugging Face Hub 上的仓库
model.push_to_hub_gguf("Hextalk4bit", tokenizer, quantization_method = "q4_k_m", token=HUGGINGFACE_TOKEN)

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.04 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [03:06<00:00,  5.82s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving Hextalk4bit/pytorch_model-00001-of-00004.bin...
Unsloth: Saving Hextalk4bit/pytorch_model-00002-of-00004.bin...
Unsloth: Saving Hextalk4bit/pytorch_model-00003-of-00004.bin...
Unsloth: Saving Hextalk4bit/pytorch_model-00004-of-00004.bin...
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at Hextalk4bit into f16 GGUF format.
The output location will be /content/Hextalk4bit/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: Hextalk4bit
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF f

  0%|          | 0/1 [00:00<?, ?it/s]

unsloth.Q4_K_M.gguf:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### We removed it in GGUF's chat template for you.


Saved GGUF to https://huggingface.co/Beyondz0110/Hextalk4bit
